In [1]:
import json
import random
import os

In [2]:
import json
with open('discipline_structure.json', 'r') as f:
    data = json.load(f)

In [3]:
data

{'agricultural sciences': {'agriculture, forestry, and fisheries': {'description': 'Agriculture; Forestry; Fishery; Soil science; Horticulture, viticulture; Agronomy, plant breeding and plant protection',
   'abstracts': ['Conclusion: Among herbicidal treatments, the highest nutrient content, nutrient uptake, productivity, net returns and benefit cost ratio in pigeon pea were observed in the pre-emergence application of pendimethalin + imazethapyr @ 1000 + 75 g ha(-1). Therefore, this treatment may be an effective and profitable alternative to the existing manual and costlier recommendation (two hoeing at 25 and 45 DAS) of weed control in pigeon pea in Haryana locality.',
    '(NH4)(2)S2O3 or its mixt. with (NH4PO3)N were added to urea-NH4NO3 fertilizer to reduce NH3 emissions. The redn. of NH3 losses was 35-39% for (NH4)(2)S2O3 and 55-70% for the addn. of (NH4)(2)S2O3-(NH4PO3)N mixt. when compared to the fertilizer without additives.',
    "The method of extracting nematodes from the 

In [4]:
prompts_directory = 'prompts'
hierarchical_folder = os.path.join(prompts_directory, 'hierarchical')
non_hierarchical_folder = os.path.join(prompts_directory, 'non-hierarchical')
os.makedirs(hierarchical_folder, exist_ok=True)
os.makedirs(non_hierarchical_folder, exist_ok=True)

## General statements

In [5]:
beginning = "Given the following {} classes: {} \n"
demand = "Return the most relevant classes for the FOLLOWING abstract. Return ONLY THE CLASSES separated by \"&\" in the ORDER OF RELEVANCE WITH THEIR RELEVANCE SCORE (between 0 and 100) in one line and never any aditional context: \n"
disclaimer = "Please, DON'T RETURN OTHER CLASSES, ONLY A SUBSET OF THE {}, otherwise the result is invalid!!! The results of my study depends on it.\n DON'T RETURN MORE THAN FIVE RESULTS. ONLY THE MOST RELEVANT AND MAX 5 (1,2,3,4,5 RESULTS)!!!! \n"
example = "Example of answer: \"{}:70&{}:60\" "
description_abstracts_intro = "The classes description and example of abstracts: \n"
description_intro = "The classes description: \n"
abstracts_intro = "The classes abstract example: \n"
title_intro = "The classes title example: \n"
description_title_intro = "The classes description and example of titles: \n"
description_abstract_class = "{}: {} \n Example abstract: {} \n"
description_title_class = "{}: {} \n Example titles: {} \n"
expl_class = "{}: {} \n"

## Level 0 hierarchical prompt

In [6]:
classes = list(data.keys())
nb_classes = len(classes)

### Basic, just enumeration

In [7]:
prompt = beginning.format(nb_classes, '; '.join(classes)) + demand + '{} \n' + disclaimer.format(nb_classes) + example.format(*random.choices(classes, k=2))
with open(os.path.join(hierarchical_folder, f"level0pred_basic.txt"), 'w+') as file:
        file.write(prompt)

Observation: with the basic level 0 prompt, the model struggles to really define each class. It might assume that history is in Social Sciences, for example

### Description formed by enumeration of disciplines belonging to that disciplinary area

In [8]:
prompt = beginning.format(nb_classes, '; '.join(classes)) + description_intro
for area, area_data in data.items():
    disciplines = list(area_data.keys())
    prompt += expl_class.format(area, ';'.join(disciplines))
prompt += demand + '{} \n' + disclaimer.format(nb_classes) + example.format(*random.choices(classes, k=2))
with open(os.path.join(hierarchical_folder, "level0pred_description.txt"), 'w+') as file:
    file.write(prompt)

## Level 1 hierarchical prompt basic (no description, no abstract)

In [9]:
folder_name = 'basic'
folder_path = os.path.join(hierarchical_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)
for area, area_data in data.items():
    disciplines = list(area_data.keys())
    num_disciplines = len(disciplines)
    prompt = beginning.format(num_disciplines, ';'.join(disciplines)) + demand + '{} \n' + disclaimer.format(num_disciplines) + example.format(*random.choices(disciplines, k=2))
    with open(os.path.join(folder_path, f"{area}.txt"), 'w+') as file:
        file.write(prompt)

## Level 1 hierarchical prompt only description

In [10]:
folder_name = 'description_only'
folder_path = os.path.join(hierarchical_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)
for area, area_data in data.items():
    disciplines = list(area_data.keys())
    num_disciplines = len(disciplines)
    explanation_prompt = description_intro
    for discipline, discipline_data in area_data.items():
        explanation_prompt += expl_class.format(discipline, discipline_data['description'])
    prompt = beginning.format(num_disciplines, ';'.join(disciplines)) + explanation_prompt + demand + '{} \n' + disclaimer.format(num_disciplines) + example.format(*random.choices(disciplines, k=2))
    with open(os.path.join(folder_path, f"{area}.txt"), 'w+') as file:
        file.write(prompt)

## Level 1 hierarchical prompt abstract only

In [11]:
folder_name = 'abstract_only'
folder_path = os.path.join(hierarchical_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)
for area, area_data in data.items():
    prompt = ''
    disciplines = list(area_data.keys())
    num_disciplines = len(disciplines)
    explanation_prompt = abstracts_intro
    for discipline, discipline_data in area_data.items():
        # Generate a string of randomly chosen titles, if available
        titles = random.choice(discipline_data['abstracts']) if discipline_data['abstracts'] else ''
        explanation_prompt += expl_class.format(discipline, titles) 
    prompt = beginning.format(num_disciplines, ';'.join(disciplines)) + explanation_prompt + demand + '{} \n' + disclaimer.format(num_disciplines) + example.format(*random.choices(disciplines, k=2))    
    with open(os.path.join(folder_path, f"{area}.txt"), 'w+') as file:
        file.write(prompt)

## Level 1 hierarchical prompt title only

In [12]:
folder_name = 'title_only'
folder_path = os.path.join(hierarchical_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)
for area, area_data in data.items():
    prompt = ''
    disciplines = list(area_data.keys())
    num_disciplines = len(disciplines)
    explanation_prompt = abstracts_intro
    for discipline, discipline_data in area_data.items():
        # Generate a string of randomly chosen titles, if available
        titles = ';'.join(random.choices(discipline_data['titles'], k=min(3, len(discipline_data['titles'])))) if discipline_data['titles'] else ''
        explanation_prompt += expl_class.format(discipline, titles) 
    prompt = beginning.format(num_disciplines, ';'.join(disciplines)) + explanation_prompt + demand + '{} \n' + disclaimer.format(num_disciplines) + example.format(*random.choices(disciplines, k=2))    
    with open(os.path.join(folder_path, f"{area}.txt"), 'w+') as file:
        file.write(prompt)

## Level 1 hierarchical prompt description + abstract

In [13]:
folder_name = 'description_abstract'
folder_path = os.path.join(hierarchical_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)
for area, area_data in data.items():
    # Initialize the prompt with an introductory description
    prompt_intro = description_abstracts_intro

    # Gather discipline names within each area and count them
    discipline_names = list(area_data.keys())
    num_disciplines = len(discipline_names)

    # Build the explanation section of the prompt for each discipline
    for discipline_name, discipline_info in area_data.items():
        abstract_sample = random.choice(discipline_info['abstracts']) if discipline_info['abstracts'] else ''
        prompt_intro += description_abstract_class.format(discipline_name, discipline_info['description'], abstract_sample)

    # Compose the full prompt
    prompt = beginning.format(num_disciplines, ';'.join(discipline_names)) + prompt_intro + demand + '{} \n' + disclaimer.format(num_disciplines) + example.format(*random.choices(discipline_names, k=2))
    
    # Write the prompt to a file within the specified folder
    with open(os.path.join(folder_path, f"{area}.txt"), 'w+') as prompt_file:
        prompt_file.write(prompt)

## Level 1 hierarchical prompt description + title

In [14]:
folder_name = 'description_title'
folder_path = os.path.join(hierarchical_folder, folder_name)
os.makedirs(folder_path, exist_ok=True)
for area, area_data in data.items():
    # Initialize the prompt with an introductory description
    prompt_intro = description_abstracts_intro

    # Gather discipline names within each area and count them
    discipline_names = list(area_data.keys())
    num_disciplines = len(discipline_names)

    # Build the explanation section of the prompt for each discipline
    for discipline_name, discipline_info in area_data.items():
        titles_sample = ';'.join(random.choices(discipline_info['titles'], k=min(3, len(discipline_info['titles'])))) if discipline_info['titles'] else ''
        prompt_intro += description_abstract_class.format(discipline_name, discipline_info['description'], titles_sample)

    # Compose the full prompt
    prompt = beginning.format(num_disciplines, ';'.join(discipline_names)) + prompt_intro + demand + '{} \n' + disclaimer.format(num_disciplines) + example.format(*random.choices(discipline_names, k=2))
    
    # Write the prompt to a file within the specified folder
    with open(os.path.join(folder_path, f"{area}.txt"), 'w+') as prompt_file:
        prompt_file.write(prompt)

## Non-hierarchical prompt basic

In [15]:
classes = []
for _, value in data.items():
    classes.extend(list(value.keys()))
    
nb_classes = len(classes)  # Calculate the total number of classes

# Prepare the prompt
prompt = beginning.format(nb_classes, ';'.join(classes)) + demand + '{} \n' + disclaimer.format(nb_classes) + example.format(*random.choices(classes, k=2))

# Write the prompt to basic.txt in the specified non_hierarchical_folder
with open(os.path.join(non_hierarchical_folder, "basic.txt"), 'w+') as file:
    file.write(prompt)

## Non-hierarchical prompt description

In [16]:
classes = []
expl_prompts = []
for area_data in data.values():
    for discipline_name, discipline_info in area_data.items():
        classes.append(discipline_name)
        expl_prompts.append(expl_class.format(discipline_name, discipline_info['description']))

# Concatenate all discipline names and their descriptions for the prompt
nb_classes = len(classes)  # Calculate the total number of classes
class_list_str = '; '.join(classes)  # Create a string of all class names separated by semicolons
expl_prompt_str = ' '.join(expl_prompts)  # Concatenate all explanation prompts

# Prepare the prompt
prompt = beginning.format(nb_classes, class_list_str) + description_intro + expl_prompt_str + demand + '{} \n' + disclaimer.format(nb_classes) + example.format(*random.choices(classes, k=2))
print(prompt)
# Write the prompt to basic.txt in the specified non_hierarchical_folder
with open(os.path.join(non_hierarchical_folder, "description.txt"), 'w+') as file:
    file.write(prompt)

Given the following 45 classes: agriculture, forestry, and fisheries; animal and dairy science; veterinary science; agricultural biotechnology; other agricultural sciences; civil engineering; electrical engineering, electronic engineering, information engineering; mechanical engineering; chemical engineering; materials engineering; medical engineering; environmental engineering; environmental biotechnology; industrial biotechnology; nano-technology; other engineering and technologies; history; archaeology; languages and linguistics; literature; philosophy and ethics; religion; arts (arts, history of arts, performing arts, music); other humanities; basic medicine; clinical medicine; health sciences; health biotechnology; other medical sciences; mathematics; computer and information sciences; physical sciences; chemical sciences; earth and related environmental sciences; biological sciences; other natural sciences; psychology; economics and business; educational sciences; sociology; law;